In [1]:
import pandas as pd
import numpy as np
import statsmodels.regression.linear_model as lm
import statsmodels.tools.tools as ct
import yfinance as yf

exMKT = excess Market Return
SMB = small minus big
HML = High minus Low (Value minus Growth)
RMW = Robust minus Weak (Robust Profitability minus Weak Profitability)
CMA = Conservative minus Aggressive

In [2]:
stock_ret = yf.download('TSLA')
stock_ret.drop(labels = ['Open','High','Low','Close','Volume'], axis = 1, inplace = True)
stock_ret.rename(columns={'Adj Close': 'Price'}, inplace = True)
ret = stock_ret.pct_change() *100
ret.dropna(inplace=True)
ret.rename(columns={'Price': 'Ret'}, inplace = True)
ret.head()

[*********************100%***********************]  1 of 1 completed


,Ret
Date,
2010-06-30,-0.251153
2010-07-01,-7.847243
2010-07-02,-12.568312
2010-07-06,-16.093751
2010-07-07,-1.924264


In [3]:
factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3_daily.csv', index_col='Date', parse_dates=True)
factors

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
1963-07-01,-0.67,0.01,-0.35,0.03,0.11,0.012
1963-07-02,0.79,-0.31,0.24,-0.08,-0.25,0.012
1963-07-03,0.63,-0.16,-0.09,0.13,-0.24,0.012
1963-07-05,0.40,0.09,-0.26,0.07,-0.28,0.012
1963-07-08,-0.63,0.07,-0.19,-0.27,0.06,0.012
...,...,...,...,...,...,...
2022-03-25,0.27,-0.29,2.17,0.19,1.33,0.000
2022-03-28,0.73,-0.88,-1.69,0.49,-1.19,0.000
2022-03-29,1.45,1.18,-1.92,-0.30,-0.97,0.000


In [4]:
factors.describe()

,Mkt-RF,SMB,HML,RMW,CMA,RF
count,14790.000000,14790.000000,14790.000000,14790.000000,14790.000000,14790.000000
mean,0.027912,0.007362,0.014692,0.013397,0.013638,0.017321
std,1.014559,0.540794,0.568965,0.388118,0.368635,0.012691
min,-17.440000,-11.230000,-4.960000,-3.020000,-5.890000,0.000000
25%,-0.410000,-0.270000,-0.230000,-0.170000,-0.180000,0.007000
50%,0.050000,0.020000,0.010000,0.010000,0.010000,0.018000
75%,0.500000,0.300000,0.250000,0.190000,0.190000,0.024000
max,11.350000,6.170000,6.750000,4.510000,2.530000,0.061000


In [5]:
factors.corr()

,Mkt-RF,SMB,HML,RMW,CMA,RF
Mkt-RF,1.000000,-0.077158,-0.148649,-0.197783,-0.348262,-0.021300
SMB,-0.077158,1.000000,0.135406,-0.265103,0.035861,-0.012343
HML,-0.148649,0.135406,1.000000,0.028988,0.544996,0.018533
RMW,-0.197783,-0.265103,0.028988,1.000000,0.102808,-0.000520
CMA,-0.348262,0.035861,0.544996,0.102808,1.000000,0.017844
RF,-0.021300,-0.012343,0.018533,-0.000520,0.017844,1.000000


In [6]:
data = pd.merge(ret, factors, on='Date')
data

,Ret,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,
2010-06-30,-0.251153,-0.98,0.00,-0.35,-0.02,-0.02,0.001
2010-07-01,-7.847243,-0.40,-0.37,-0.42,0.48,0.13,0.001
2010-07-02,-12.568312,-0.50,-0.33,-0.45,0.10,-0.36,0.001
2010-07-06,-16.093751,0.33,-1.96,0.11,0.03,-0.44,0.001
2010-07-07,-1.924264,3.17,0.06,0.43,-0.74,0.94,0.001
...,...,...,...,...,...,...,...
2022-03-25,-0.323494,0.27,-0.29,2.17,0.19,1.33,0.000
2022-03-28,8.034508,0.73,-0.88,-1.69,0.49,-1.19,0.000
2022-03-29,0.707977,1.45,1.18,-1.92,-0.30,-0.97,0.000


In [7]:
data['exRet'] = data['Ret']-data['RF']
factor_names = ['Mkt-RF','SMB','HML','RMW','CMA']
factor_names

['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

In [8]:
ff5fm = lm.OLS(data['exRet'], data[factor_names], hasconst=False).fit()

In [9]:
ff5fm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  exRet   R-squared (uncentered):                   0.234
Model:                            OLS   Adj. R-squared (uncentered):              0.233
Method:                 Least Squares   F-statistic:                              180.8
Date:                Sun, 22 May 2022   Prob (F-statistic):                   2.78e-168
Time:                        16:49:37   Log-Likelihood:                         -7570.1
No. Observations:                2960   AIC:                                  1.515e+04
Df Residuals:                    2955   BIC:                                  1.518e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         1.2205      0.057     21.573      0.000       1.110       1.331
SMB            0.5048      0.103      4.880      0.000       0.302       0.708
HML           -0.6319      0.099     -6.388      0.000      -0.826      -0.438
RMW           -0.6773      0.147     -4.617      0.000      -0.965      -0.390
CMA           -0.7545      0.201     -3.750      0.000      -1.149      -0.360
==============================================================================
Omnibus:                      577.079   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6407.974
Skew:                           0.592   Prob(JB):                         0.00
Kurtosis:                      10.110   Cond. No.                         4.18
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""